<a href="https://colab.research.google.com/github/sulavpradhan/face-recognition-/blob/main/faceRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version


Python 3.7.10


In [ ]:
!pip3 install mtcnn
!pip3 install opencv-contrib-python
!pip install --user keras
!pip install Keras-Applications
!pip install git+https://github.com/rcmalli/keras-vggface.git

     |████████████████████████████████| 2.3MB 7.5MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-f_c0yvoi
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-f_c0yvoi
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp37-none-any.whl size=8326 sha256=07d39037b7c295dc0cda0a16fbdbcc83f76bc1ee9f47f2fd4fe03923577c0f2b
  Stored in directory: /tmp/pip-ephem-wheel-cache-er6wnljp/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import mtcnn
print (mtcnn.__version__)

0.1.0


In [ ]:
from keras_vggface.vggface import VGGFace

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
import matplotlib.pyplot as plt
from numpy import expand_dims
import numpy as np
from matplotlib import pyplot
from numpy import asarray
from PIL import Image
from mtcnn.mtcnn import MTCNN
import cv2
from keras import applications
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions
import os
from keras.models import load_model
from scipy.spatial.distance import cosine
import glob



model = MTCNN()

In [ ]:
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array


In [ ]:
# create a vggface2 model
model = VGGFace(model='vgg16')
# summarize input and output shape
print('Inputs: %s' % model.inputs)
print('Outputs: %s' % model.outputs)

580083712/580070376 [==============================] - 18s 0us/step
Inputs: [<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_1')>]
Outputs: [<KerasTensor: shape=(None, 2622) dtype=float32 (created by layer 'fc8/softmax')>]


In [ ]:
def get_embeddings(filenames):
	# extract faces
	faces = [extract_face(f) for f in filenames]
	# convert into an array of samples
	samples = asarray(faces, 'float32')
	# prepare the face for the model, e.g. center pixels
	samples = preprocess_input(samples, version=2)
	# create a vggface model
	model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
	# perform prediction
	yhat = model.predict(samples)
	return yhat

In [ ]:
def is_match(known_embedding, candidate_embedding, thresh=0.5):
	# calculate distance between embeddings
	score = cosine(known_embedding, candidate_embedding)
	if score <= thresh:
		return 1
	else:
		return 0

In [ ]:
filenames = sorted(glob.glob("/content/drive/MyDrive/Colab Notebooks/images/*"))
print (filenames)



['/content/drive/MyDrive/Colab Notebooks/images/bradd.jpg', '/content/drive/MyDrive/Colab Notebooks/images/googleface.jpg', '/content/drive/MyDrive/Colab Notebooks/images/ruja.jpg', '/content/drive/MyDrive/Colab Notebooks/images/sabudh.jpg']


In [ ]:
embeddings = get_embeddings(filenames)

94707712/94694792 [==============================] - 2s 0us/step


In [ ]:
userinput = input("enter the name of the image path: ")
print (userinput)
imageFileName=[userinput]
new = get_embeddings(imageFileName)
print (new)


enter the name of the image path: /content/drive/MyDrive/Colab Notebooks/unseen face/ACE074BCT087.jpg
/content/drive/MyDrive/Colab Notebooks/unseen face/ACE074BCT087.jpg
[[0.00608063 0.8952685  0.31616887 ... 1.819069   0.         0.0967327 ]]


In [ ]:
embedding = np.array(embeddings)
np.save('embedding',embedding)
embd=np.load('embedding.npy')
embd_keys=['brad', 'google face', 'ruja', 'sabudh' ]
res = {embd_keys[i]: embedding[i] for i in range(len(embd_keys))} 
print ("Resultant dictionary is : " +  str(res)) 
i=0
e=0
for x in embd:
  print (x)
  e = is_match(x,new)
  if e == 1:
    #print ('is a match with ')  
    break
  i += 1
#if e == 0 :
  #print ('the person is not recognized')
#key = embd.keys(i)
if e == 1:
  k = 0
  for j in res.keys():
    if k == i:
      key = j 
    k +=1
   

  print (key)



# load image from file
  pixels = pyplot.imread(userinput)
# create the detector, using default weights
  detector = MTCNN()
# detect faces in the image
  results = detector.detect_faces(pixels)
# extract the bounding box from the first face
  x1, y1, width, height = results[0]['box']
  x2, y2 = x1 + width, y1 + height
  img=cv2.imread(userinput)
  gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0), 2)
  name = key
  img=cv2.putText(img,name,(x1,y1-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,255),2,cv2.LINE_AA)
  cv2_imshow(img)
  cv2.waitKey(0)
  cv2.destroyAllWindows()


if e == 0 :
  print ('the person is not recognized')

  


Resultant dictionary is : {'brad': array([0.08709431, 0.        , 0.25766683, ..., 0.        , 1.6011869 ,
       0.        ], dtype=float32), 'google face': array([0.34000015, 9.297607  , 0.        , ..., 2.6449475 , 1.2109889 ,
       0.757356  ], dtype=float32), 'ruja': array([0.        , 0.4449838 , 0.89449984, ..., 0.18244961, 1.3435259 ,
       5.2941422 ], dtype=float32), 'sabudh': array([0.        , 0.13957237, 7.0253463 , ..., 1.0670841 , 1.951842  ,
       0.        ], dtype=float32)}
[0.08709431 0.         0.25766683 ... 0.         1.6011869  0.        ]
[0.34000015 9.297607   0.         ... 2.6449475  1.2109889  0.757356  ]
[0.         0.4449838  0.89449984 ... 0.18244961 1.3435259  5.2941422 ]
[0.         0.13957237 7.0253463  ... 1.0670841  1.951842   0.        ]
the person is not recognized
